In [ ]:
#imports 

from stellargraph import StellarGraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from tensorflow import keras

Metal device set to: Apple M1 Pro


2022-09-29 16:40:43.892988: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-29 16:40:43.893315: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
combined_phenotype = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Metadata/phenotypic_data.csv'
labels_mappings= pd.read_csv(combined_phenotype)
labels_mappings

,Subject,Subject Type,Label
0,40013,Control,0
1,40014,Control,0
2,40017,Control,0
3,40018,Control,0
4,40019,Control,0
...,...,...,...
416,2168,Patient,1
417,2169,Patient,1
418,2170,Patient,1
419,2171,Patient,1


In [ ]:
def row_transform(arr, threshold):
  for i in range(len(arr)):
      arr[i] = arr[i] if arr[i]> threshold else abs(arr[i])+0.0001
  return arr

def binarize(df, threshold):
    df = df.transform(lambda x: row_transform(x, threshold))

In [ ]:
uw = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Weighted v2'

In [ ]:
dirs=[uw]
count = 0 

In [ ]:
graphs = list()
graph_labels = list()
sub_lst = list(range(0,164))

In [ ]:
for dir in dirs:
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            subject = file[14:18]
            df = pd.read_csv(dir+'/'+file, header=None)
            binarize(df,0)
            df = df.replace(np.nan ,0)
            G = nx.from_pandas_adjacency(df)

            #adding other attributes
            wd = dict(G.degree(weight='weight'))
            cv = nx.closeness_vitality(G, node=None, weight="weight", wiener_index=None)
            andeg = nx.average_neighbor_degree(G, weight="weight")
            cons = nx.constraint(G, nodes=None, weight="weight")
            es = nx.effective_size(G, nodes=None, weight="weight")
            
            node_features_data=list(zip(wd.values(),cv.values(),andeg.values(),cons.values(),es.values()))
            node_features_columns= ['weighted degree','closeness vitality','avg neighbor degree',
                                    'constraint','effective size']
            node_data= pd.DataFrame(node_features_data, index=sub_lst, columns=node_features_columns)
            
            #saving individual node features
            csv_path='weighted_node_features_'+str(subject)+'.csv'
            node_data.to_csv(csv_path)
            
            count+=1
            print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
